In [ ]:
1 - структура БД - названия, исходные данные, нормальные данные(?The difference?) - ссылки на полные версии статей
2 - веб-сервис (фласк) - произвольная фраза, ссылки на метариал, где встречается, кусочек страницы с ней

1 - где брать и искать - из csv
тексты придется искать по папкам

2 - фласк, html - искать фразу по каждому тексту?
оформление кусочка текста статьи....

лемматизированный
запрос
лемм запрос
лемм текст
ответ просто

срезы

подсвечивание нужного слова (?)

In [65]:
# база данных
# названия статей, тексты  Plain, mystem-тексты, ссылки на статьи
import sqlite3
import csv
import re

conn = sqlite3.connect('data.db')
c = conn.cursor()
c.execute('CREATE TABLE IF NOT EXISTS articles(name text, plain text, mystem text, link text)')
# c.execute('INSERT INTO articles VALUES (?, ?, ?, ?)', ()) #здесь должен быть словарь? создать словарь из файла метаданных
D = {}
n = 0
with open('.\YV\metadata.csv', 'r', encoding='utf-8') as csvfile:
    filereader = csv.reader(csvfile, delimiter='\t')
    for row in filereader:
        d = {}
        n += 1
        d['plain_path'] = row[0]
        with open(d['plain_path'], 'r', encoding='utf-8') as f:
            text = f.readlines()
            d['plain'] = (''.join(text[5:]))        
        d['name'] = row[2]
        d['link'] = row[10]
        d['mystem_path'] = d['plain_path'].replace('plain', 'mystem-plain')
        with open(d['mystem_path'], 'r', encoding='utf-8') as f:
            d['mystem'] = f.read()
        D[n] = d
# + сокращение!!!! (это только для выдачи :)
for key, value in D.items():
#     for key in item.items():
    c.execute('INSERT INTO articles VALUES (?, ?, ?, ?)', (D[key]['name'], D[key]['plain'], D[key]['mystem'], D[key]['link']))
conn.commit()
conn.close()

#works! 3
# база данных работает

In [10]:
# поисковик
from flask import Flask
from flask import url_for, render_template, request, redirect


app = Flask(__name__)

# страница поиска
@app.route('/')
def search():
    return render_template('search.html')
   
# страница результатов
@app.route('/results')
def results():
    if request.args:
        query = request.args['query']
        links = query_in_text(query_to_mystem(query))
        return render_template('results.html', query = query, links = links)
    return redirect(url_for('search'))
        
if __name__ == '__main__':
    app.run(debug=True)
    

SyntaxError: unexpected EOF while parsing (<ipython-input-10-ffe2a37fbfd6>, line 17)

In [24]:
# функция лемматизации запроса
from pymystem3 import Mystem

def query_to_mystem(query):
    text = query
    m = Mystem()
    lemmas = m.lemmatize(text)

    return lemmas
    
query_to_mystem('белую собаку')

['белый', ' ', 'собака', '\n']

In [69]:
# функция поиска лемматизированного запроса в лемматизированном тексте
import re
import sqlite3


def query_in_text(lemmas):
    symbols = '(),"'''
    links = []
    s = len(lemmas)
    k = ''
    for i in range(0, s):
        if lemmas[i]!= '\n' and lemmas[i] != ' ':
            k += lemmas[i] + '=' + '.+?'
    conn = sqlite3.connect('datag.db')
    c = conn.cursor()
    c.execute('SELECT mystem FROM articles LIMIT 10')
    a = c.fetchall()    
    for item in a:
        b = ''.join(str(item)) 
        r = re.search(k, b)
        if r:
            c.execute('SELECT link FROM articles WHERE mystem=?', item)
#             e = c.fetchone()
            e = str(c.fetchone()).strip(symbols).strip("''")
#             g = str(e.strip(symbols))
            links.append(e)
#     print(links)
    return links            

# вроде работает....
query_in_text(query_to_mystem('россия'))

['http://www.vecherniy.com/wall?id=1',
 'http://www.vecherniy.com/wall?id=3',
 'http://www.vecherniy.com/wall?id=7',
 'http://www.vecherniy.com/wall?id=13']

In [16]:
# c.execute('INSERT INTO articles VALUES (?, ?, ?, ?)', ()) #здесь должен быть словарь? создать словарь из файла метаданных
import sqlite3

D = []

with open('.\YV\metadata.csv', 'r', encoding='utf-8') as csvfile:
    filereader = csv.reader(csvfile, delimiter='\t')
    for row in filereader:
        d = {}
#         print(row[0])
        d['plain_path'] = row[0]
        with open(d['plain_path'], 'r', encoding='utf-8') as f:
            text = f.readlines()
            d['plain'] = (''.join(text[5:]))        
        d['name'] = row[2]
        d['link'] = row[10]
        d['mystem_path'] = d['plain_path'].replace('plain', 'mystem-plain')
        with open(d['mystem_path'], 'r', encoding='utf-8') as f:
            d['mystem'] = f.read()
        D.append(d)
# + сокращение!!!! (это только для выдачи :)
for item in D:
#     print(item)
#     print('\n')
    for key, value in item.items():
        print(key, value)
        print('\n')
#         print(value)
# print(D)
        

plain_path YV\plain\2015\03\article1.txt


plain В последние годы Россия приступила к активному хозяйственному освоению своих северных территорий, включая добычу углеводородов, а также развитию Северного морского пути, который все в большей степени становится альтернативой традиционным маршрутам из Европы в Азию. На защиту российских интересов в Арктике направлен целый комплекс мер, в том числе военного характера, учитывая возросшее внимание к региону со стороны стран-членов НАТО. В апреле 2014 года президент Владимир Путин объявил о создании в российской части Арктики единой системы базирования надводных кораблей и подлодок нового поколения. По его словам, также предстоит повысить надежность охраны российской границы в Арктике, в том числе за счет усиления морской составляющей пограничной группировки. Во исполнение Указа Президента от 3 февраля 2015 года № 50 «О Государственной комиссии по вопросам развития Арктики» Правительство Российской Федерации приняло постановление о Государств

In [71]:
def query_in_text(query):
    symbols = '(),"'''
    m = Mystem()
    lemmas = m.lemmatize(query)
    s = len(lemmas)
    #список ссылок на статьи
    links = []
    #регулярное выражение, которое поможет в поиске слова/фразы
    k = ''
    for i in range(0, s):
        if lemmas[i]!= '\n' and lemmas[i] != ' ':
            k += lemmas[i] + '=' + '.+?'
    conn = sqlite3.connect('datah.db')
    c = conn.cursor()
    #выберем все mуstem-тексты
    c.execute('SELECT mystem FROM articles')
    a = c.fetchall()    
    for item in a:
        #отдельный mуstem-текст        
        b = ''.join(str(item)) 
        r = re.search(k, b)
        #если регулярное выражение в тексте, вытаскиваем ссылку статьи        
        if r:
            c.execute('SELECT link FROM articles WHERE mystem=?', item)
            e = str(c.fetchone()).strip(symbols).strip("''")
            if e not in links:
                links.append(e)
    conn.commit()
    conn.close()
    return links 

print(query_in_text('россия'))

['http://www.vecherniy.com/wall?id=1', 'http://www.vecherniy.com/wall?id=3', 'http://www.vecherniy.com/wall?id=7', 'http://www.vecherniy.com/wall?id=13', 'http://www.vecherniy.com/wall?id=19', 'http://www.vecherniy.com/wall?id=23']
